# 1. 単純ベイズ分類器

### パッケージの読み込み

ここではガウス分布の単純ベイズ分類器（GaussianNB）を利用。単純ベイズ分類器を含む機械学習パッケージ（scikit-learn）については資料を参照のこと。

In [2]:
import numpy as np
from sklearn.naive_bayes import GaussianNB

### 学習セットの用意

学習セットの説明については資料を参照のこと。

In [3]:
X = np.array([[1, 0, 1, 1, 2, 1, 1, 0, 0],
              [1, 2, 0, 1, 0, 0, 3, 3, 1],
              [2, 0, 1, 0, 3, 1, 0, 2, 1]])
y = np.array([1, 2, 3])

### 単純ベイズ分類器の学習

In [4]:
clf = GaussianNB()

In [5]:
clf.fit(X, y)

GaussianNB()

In [6]:
clf.score(X, y)

1.0

### 学習済の分類器を用いた推定

In [7]:
t = np.array([[1, 0, 1, 1, 0, 0, 1, 0, 0]])
clf.predict(t)

array([1])

# 2. テキストの分類

### パッケージの読み込み

CountVectorizerは単語の数を数えるパッケージで，これを利用してBoWベクトルを作成します。

In [8]:
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import CountVectorizer

### 学習セットの用意

In [9]:
s = ['今日 は とても 天気 が いい',
     '今日 は 晴れ です',
     '天気 が いい 日 は 晴れ です']

In [10]:
vectorizer = CountVectorizer()
vectorizer.fit(s)
X = vectorizer.transform(s).toarray()

In [11]:
y = np.array([1, 2, 3])

### 単純ベイズ分類器の学習

In [12]:
clf = GaussianNB()

In [13]:
clf.fit(X, y)

GaussianNB()

In [14]:
clf.score(X, y)

1.0

### 学習済分類器を用いた推定

In [15]:
t = vectorizer.transform(["明日 は 天気 が いい"]).toarray()
clf.predict(t)

array([1])

# 3. 長いテキストの分類

### janome（形態素解析）のインストール

In [16]:
!pip install janome

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 19.8 MB/s eta 0:00:00


### パッケージの読み込み

In [17]:
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import CountVectorizer
from janome.tokenizer import Tokenizer

### 学習用文書の読み込み
7つアップしたうち，4つを学習用に，1つはテスト用にします。

In [18]:
# テキスト読み込み用の関数定義
def file_load(file_name):
  text = None
  with open(file_name, "r") as f:
    text = "\n".join(f.readlines())
  return text

In [21]:
file_list = ['bot_content.txt','sore_content.txt', 'kappa_content.txt','shayou_content.txt']
text_list = []
for file in file_list:
  text_list.append(file_load(file))

In [24]:
# 最初のテキスト（坊っちゃん）の最初の1000文字を表示
text_list[0][0:1000]

'<br/>\n\n<div class="jisage_5" style="margin-left: 5em"><h4 class="naka-midashi"><a class="midashi_anchor" id="midashi10">一</a></h4></div>\n\n<br/>\n\n\u3000<ruby><rb>親譲</rb><rp>（</rp><rt>おやゆず</rt><rp>）</rp></ruby>りの<ruby><rb>無鉄砲</rb><rp>（</rp><rt>むてっぽう</rt><rp>）</rp></ruby>で小供の時から損ばかりしている。小学校に居る時分学校の二階から飛び降りて一週間ほど<ruby><rb>腰</rb><rp>（</rp><rt>こし</rt><rp>）</rp></ruby>を<ruby><rb>抜</rb><rp>（</rp><rt>ぬ</rt><rp>）</rp></ruby>かした事がある。なぜそんな<ruby><rb>無闇</rb><rp>（</rp><rt>むやみ</rt><rp>）</rp></ruby>をしたと聞く人があるかも知れぬ。別段深い理由でもない。新築の二階から首を出していたら、同級生の一人が<ruby><rb>冗談</rb><rp>（</rp><rt>じょうだん</rt><rp>）</rp></ruby>に、いくら<ruby><rb>威張</rb><rp>（</rp><rt>いば</rt><rp>）</rp></ruby>っても、そこから飛び降りる事は出来まい。弱虫やーい。と<ruby><rb>囃</rb><rp>（</rp><rt>はや</rt><rp>）</rp></ruby>したからである。<ruby><rb>小使</rb><rp>（</rp><rt>こづかい</rt><rp>）</rp></ruby>に負ぶさって帰って来た時、おやじが大きな<ruby><rb>眼</rb><rp>（</rp><rt>め</rt><rp>）</rp></ruby>をして二階ぐらいから飛び降りて腰を抜かす<ruby><rb>奴</rb><rp>（</rp><rt>やつ</rt><rp>）</rp></ruby>があるかと<ruby><rb>云</rb><rp>（</rp><rt>い</rt><rp>

### クレンジング

まずクレンジング用の関数を定義して，そのあとクレンジング

In [25]:
import re

def cleansing(text):
  clean_text = re.sub("\s", "", text)                     # 余分な空白（改行や字下げの空白）を除去
  clean_text = clean_text.replace("<br/>", "\n")          # <br/>タグを改行に変換
  clean_text = re.sub(r"<rp>[^<]+</rp>", "", clean_text)  # ルビの前後の括弧を除去
  clean_text = re.sub(r"<rt>[^<]+</rt>", "", clean_text)  # ルビのテキストを除去
  clean_text = re.sub(r"<[^>]+>", "", clean_text)         # それ以外のタグを除去
  return clean_text

In [26]:
clean_text_list = []
for text in text_list:
  clean_text_list.append(cleansing(text))

In [27]:
# 最初のテキストの最初の1000文字を表示
clean_text_list[0][0:1000]

'\n一\n親譲りの無鉄砲で小供の時から損ばかりしている。小学校に居る時分学校の二階から飛び降りて一週間ほど腰を抜かした事がある。なぜそんな無闇をしたと聞く人があるかも知れぬ。別段深い理由でもない。新築の二階から首を出していたら、同級生の一人が冗談に、いくら威張っても、そこから飛び降りる事は出来まい。弱虫やーい。と囃したからである。小使に負ぶさって帰って来た時、おやじが大きな眼をして二階ぐらいから飛び降りて腰を抜かす奴があるかと云ったから、この次は抜かさずに飛んで見せますと答えた。\n親類のものから西洋製のナイフを貰って奇麗な刃を日に翳して、友達に見せていたら、一人が光る事は光るが切れそうもないと云った。切れぬ事があるか、何でも切ってみせると受け合った。そんなら君の指を切ってみろと注文したから、何だ指ぐらいこの通りだと右の手の親指の甲をはすに切り込んだ。幸ナイフが小さいのと、親指の骨が堅かったので、今だに親指は手に付いている。しかし創痕は死ぬまで消えぬ。\n庭を東へ二十歩に行き尽すと、南上がりにいささかばかりの菜園があって、真中に栗の木が一本立っている。これは命より大事な栗だ。実の熟する時分は起き抜けに背戸を出て落ちた奴を拾ってきて、学校で食う。菜園の西側が山城屋という質屋の庭続きで、この質屋に勘太郎という十三四の倅が居た。勘太郎は無論弱虫である。弱虫の癖に四つ目垣を乗りこえて、栗を盗みにくる。ある日の夕方折戸の蔭に隠れて、とうとう勘太郎を捕まえてやった。その時勘太郎は逃げ路を失って、一生懸命に飛びかかってきた。向うは二つばかり年上である。弱虫だが力は強い。鉢の開いた頭を、こっちの胸へ宛ててぐいぐい押した拍子に、勘太郎の頭がすべって、おれの袷の袖の中にはいった。邪魔になって手が使えぬから、無暗に手を振ったら、袖の中にある勘太郎の頭が、右左へぐらぐら靡いた。しまいに苦しがって袖の中から、おれの二の腕へ食い付いた。痛かったから勘太郎を垣根へ押しつけておいて、足搦をかけて向うへ倒してやった。山城屋の地面は菜園より六尺がた低い。勘太郎は四つ目垣を半分崩して、自分の領分へ真逆様に落ちて、ぐうと云った。勘太郎が落ちるときに、おれの袷の片袖がもげて、急に手が自由になった。その晩母が山城屋に詫びに行ったついでに袷の片袖も取り返して来た。\nこの外いたずらは大分やった。大工の兼公と肴

### 分かち書きにする

In [28]:
# 分かち書き用の関数定義
def wakati_text(text, pos=["名詞", "動詞"]):
    tokenizer = Tokenizer()
    doc = tokenizer.tokenize(text)
    wakati = None
    word_list = []
    for token in doc:
        p = token.part_of_speech.split(",")[0]
        if p in pos:
            word_list.append(token.base_form)
    if 0 < len(word_list):
        wakati = " ".join(word_list)
    return wakati

In [29]:
wakati_list = []
for clean_text in clean_text_list:
    wakati_list.append(wakati_text(clean_text, ["名詞", "形容動詞", "形容詞"]))

In [31]:
# 最初のテキストデータの最初の1000文字を表示
wakati_list[0][0:1000]

'一 親譲り 無鉄砲 供 時 損 小学校 時分 学校 二 階 一 週間 腰 事 無闇 人 深い 理由 ない 新築 二 階 首 同級生 一 人 冗談 そこ 事 弱虫 ー い 小使 時 おやじ 眼 二 階 腰 奴 次 親類 もの 西洋 製 ナイフ 奇麗 刃 日 友達 一 人 事 そう ない 事 何 そん 君 指 注文 指 通り 右 手 親指 甲 幸 ナイフ 小さい の 親指 骨 堅い 今 親指 手 創痕 庭 東 二 十 歩 南 上がり 菜園 真中 栗 木 一 本 これ 命 大事 栗 実 熟 時分 起き抜け 背戸 奴 学校 菜園 西側 山城 屋 質屋 庭 続き 質屋 勘太郎 十 三 四 倅 勘太郎 弱虫 弱虫 癖 四つ 目 垣 栗 日 夕方 折戸 蔭 勘太郎 時 勘太郎 路 一生懸命 向う 二つ 年上 弱虫 力 強い 鉢 頭 こっち 胸 拍子 勘太郎 頭 おれ 袷 袖 中 邪魔 手 手 袖 中 勘太郎 頭 右左 苦しい 袖 中 おれ 二の腕 痛い 勘太郎 垣根 足 搦 向う 山城 屋 地面 菜園 六 尺 がた 低い 勘太郎 四つ 目 垣 半分 自分 領分 真逆様 ぐうと 勘太郎 とき おれ 袷 袖 急 手 自由 晩 母 山城 屋 詫び ついで 袷 袖 外 いたずら 大工 兼 公 肴 屋 角 茂作 人参 畠 事 人参 芽 処 藁 一 面 上 三 人 半日 相撲 人参 みんな 古川 田圃 井戸 尻 事 太い 孟宗 節 深い 中 水 そこ ら 稲 仕掛 時分 仕掛 石 棒 井戸 中 水 の うち 飯 古川 真赤 罰金 よう おやじ おれ 母 兄 贔屓 兄 や 色 白い 芝居 真似 女形 の 好き おれ 度 こいつ 碌 もの おやじ 乱暴 乱暴 行く先 母 碌 もの ご覧 通り 始末 行く先 の 無理 懲役 母 病気 二 三 日 前 台所 宙返り 角 肋骨 痛い 母 お前 よう もの 顔 親類 泊り 報知 早い 大病 大人しい よい 例 兄 おれ 親不孝 おれ ため おっかさん 早い ん 口惜しい 兄 横っ面 大変 母 おやじ 兄 三 人 おやじ 何 男 人 顔 貴様 駄目 駄目 口癖 よう 何 駄目 ん 今 妙 おやじ もん 兄 実業 家 英語 勉強 元来 女 よう 性分 ずるい 仲 よい 十 日 一 遍 割 喧嘩 時 将棋 卑怯 待 駒 人 嬉しい そう 腹 手 飛車 眉間 

### BoWの計算

In [32]:
vectorizer = CountVectorizer()
vectorizer.fit(wakati_list)
X = vectorizer.transform(wakati_list).toarray()

### テキストごとのクラスを用意
1：夏目漱石　2：芥川竜之介　3:太宰治

In [33]:
y = np.array([1, 1, 2, 3])

### 単純ベイズ分類器の学習

In [34]:
clf = GaussianNB()

In [35]:
clf.fit(X, y)

GaussianNB()

In [36]:
clf.score(X, y)

1.0

### テスト用文書の読み込み

In [38]:
test_file_list = ['34ro_content.txt', 'jigokuhen_content.txt', 'ningen_content.txt']
test_text_list = []
for file in test_file_list:
    test_text_list.append(file_load(file))

### クレンジング

In [41]:
test_clean_list = []
for text in test_text_list:
  test_clean_list.append(cleansing(text))

### テスト用文書の分かち書き

In [43]:
test_wakati_list = []
for clean_text in test_clean_list:
  test_wakati_list.append(wakati_text(clean_text, ["名詞", "形容動詞", "形容詞"]))

In [44]:
test_wakati_list[0][0:1000]

'一 目 女 隣 じいさん 話 じいさん 前 前 駅 いなか者 発車 ぎわに 頓狂 声 肌 背中 灸 あと 三四郎 記憶 じいさん 汗 肌 女 隣 腰 注意 女 京都 相乗り 時 三 四 郎 目 一色 黒い 三四郎 九州 山陽 線 京 大阪 うち 女 色 白い 故郷 よう 哀れ 女 車 室 時 異性 味方 心持ち 女 色 九州 色 三輪田 光 さん 色 国 光 さん うるさい 女 そば の ありがたい 光 さん よう の 悪い 顔 ちか ら 女 ほう 上等 口 目 額 光 さん よう だだっ広い いい 心持ち 三四郎 五 分 一 度 目 女 方 女 自分 目 こと じいさん 女 隣 腰 時 注意 長い あいだ 女 様子 時 女 おかけ じいさん 席 それ 三四郎 眠い の あいだ 女 じいさん 懇意 話 もの 目 三四郎 二 人 話 女 こと 子供 玩具 広島 京都 ほう 安い いい もの 京都 ちょっと 用 ついで 蛸薬師 そば 玩具 久しぶり 国 子供 の うれしい 夫 仕送り しかた ない 親 里 の 心配 夫 呉 海軍 職工 戦争 中 旅順 方 戦争 あっち ほう 金 大連 はじめ うち 音信 月々 もの よい 半年 前 手紙 金 不実 性質 ない 大丈夫 いつ わけ 安否 しかた ない 里 つもり じいさん 蛸薬師 玩具 興味 ない はじめ うち 返事 旅順 以後 急 同情 それ 気の毒 自分 子 戦争 中 兵隊 あっち 戦争 なん ため もの あと 景気 大事 子 物価 高い もの 世 いい 時分 もの ない みんな 戦争 おかげ 信心 大切 違い じいさん 事 女 汽車 大事 女 挨拶 元気 じいさん 者 四 人 の 一 人 ない もと 客車 ない の 急 寂しい 日 せい 駅 夫 屋根 上 灯 ランプ 三四郎 よう 前 停車場 弁当 車 二 分 ころ 例 女 三 四 郎 横 車 室 外 時 女 帯 色 三四郎 目 三四郎 鮎 びたし 頭 まま 女 後姿 便所 ん 女 ［＃「 底本 今度 正面 三四郎 弁当 がけ 下 一生懸命 箸 二 口 三 口 女 元 席 目 正面 三 四 郎 目 女 ただ 三 四 郎 横 自分 座 ところ 前 からだ 横 窓 首 静か 外 風 強い 鬢 ところ 三 四 郎 目 時 三 四 郎 弁当 折 窓 女 窓 三四郎 窓 一 

### BoWの計算

In [45]:
T = vectorizer.transform(test_wakati_list).toarray()

### 学習済の分類器を用いて推定

In [46]:
clf.predict(T)

array([1, 1, 1])